In [ ]:
# Step 1: Install Required Libraries
# You may need to install these packages first if you haven't already:
# pip install nltk scikit-learn Flask

import nltk
import random
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from flask import Flask, request, jsonify

nltk.download('punkt')

# Step 2: Prepare Data
# Sample training data for the chatbot
data = {
    "intents": [
        {"tag": "greeting", "patterns": ["Hi", "Hello", "How are you?", "Is anyone there?", "Good day"], "responses": ["Hello!", "Hi there!", "Greetings!", "How can I assist you?"]},
        {"tag": "order_status", "patterns": ["Where is my order?", "Check my order status", "Track my order"], "responses": ["Your order is on the way!", "It is expected to arrive tomorrow."]},
        {"tag": "return_policy", "patterns": ["What is your return policy?", "How do I return a product?", "Can I return my order?"], "responses": ["You can return your order within 30 days of purchase."]},
        {"tag": "goodbye", "patterns": ["Bye", "See you later", "Goodbye"], "responses": ["Goodbye!", "Have a nice day!", "See you later!"]},
        {"tag": "thanks", "patterns": ["Thanks", "Thank you", "That's helpful"], "responses": ["You're welcome!", "Glad to help!"]},
    ]
}

# Preprocess the data
intents = data['intents']
patterns = []
tags = []
responses = {}

for intent in intents:
    for pattern in intent['patterns']:
        patterns.append(pattern)
        tags.append(intent['tag'])
    responses[intent['tag']] = intent['responses']

# Step 3: Build the Rule-Based Chatbot
def rule_based_chatbot(message):
    for intent in intents:
        for pattern in intent['patterns']:
            if pattern.lower() in message.lower():
                return random.choice(intent['responses'])
    return "I'm sorry, I didn't understand that."

# Step 4: Build the Machine Learning Model
X_train, X_test, y_train, y_test = train_test_split(patterns, tags, test_size=0.2, random_state=42)

# Vectorize the text data
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)

# Train a Naive Bayes classifier
model = MultinomialNB()
model.fit(X_train_vec, y_train)

# Function to predict intent
def predict_intent(message):
    message_vec = vectorizer.transform([message])
    predicted_tag = model.predict(message_vec)[0]
    return random.choice(responses[predicted_tag])

# Step 5: Integrate with Flask
app = Flask(__name__)

@app.route('/chatbot', methods=['POST'])
def chatbot_response():
    user_message = request.json['message']
    
    # First, check the rule-based responses
    response = rule_based_chatbot(user_message)
    
    # If no response, use the machine learning model
    if "I'm sorry" in response:
        response = predict_intent(user_message)

    return jsonify({'response': response})

if __name__ == "__main__":
    app.run(port=5000)